In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000    126   10    9  112.39  535  112.12  0.26  0.01  104.56  496  103.92   
10000    89   20   19  111.65  503  110.97  0.68  0.01  103.94  463  102.73   
15000    98   30   29  111.24  484  110.28  0.96  0.01  103.67  447  102.14   
20000    90   40   39  110.98  471  109.80  1.17  0.00  103.56  438  101.84   
25000    98   50   49  110.79  462  109.47  1.31  0.00  103.40  431  101.61   
30000   103   60   59  110.65  456  109.22  1.42  0.01  103.39  428  101.49   
35000   108   70   69  110.53  450  109.01  1.52  0.01  103.26  421  101.28   
40000   103   80   79  110.44  446  108.84  1.60  0.00  103.16  416  101.08   
45000   110   90   89  110.36  442  108.69  1.66  0.01  103.14  413  101.01   
50000   111  100   99  110.28  438  108.56  1.72  0.00  103.12  411  100.97   
55000   109  110  109  110.22  436  108.45  1.77  0.00  103.03  408  100.83   
60000   102  120  119  110.17  433  108.35  1.82  0.00  103.08  408  100.88   
65000   111  130  129  110.12  431  108.26  1.85  0.00  103.05  408  100.85   
70000   124  140  139  110.08  429  108.19  1.89  0.00  103.01  407  100.80   
75000   134  150  149  110.05  427  108.12  1.92  0.00  103.03  407  100.80   
80000   104  160  159  110.01  426  108.06  1.94  0.00  103.04  407  100.80   
85000   106  170  169  109.98  424  108.01  1.97  0.00  103.03  407  100.78   
90000   108  180  179  109.95  423  107.96  1.99  0.00  103.02  406  100.77   
95000   106  190  189  109.93  422  107.92  2.01  0.00  103.05  408  100.80   
100000  110  200  199  109.91  421  107.88  2.03  0.00  103.02  406  100.76   
105000  108  210  209  109.89  420  107.84  2.05  0.00  103.01  406  100.74   
110000  106  220  219  109.87  419  107.81  2.07  0.00  102.98  404  100.71   
115000  107  230  229  109.85  418  107.77  2.08  0.00  102.99  404  100.70   
120000  106  240  239  109.84  418  107.74  2.09  0.00  102.99  404  100.69   
125000  106  250  249  109.82  417  107.72  2.11  0.00  102.96  403  100.67   
130000  106  260  259  109.81  416  107.69  2.12  0.00  102.95  403  100.66   
135000  105  270  269  109.79  416  107.66  2.13  0.00  102.94  403  100.65   
140000  106  280  279  109.78  415  107.64  2.14  0.00  102.89  401  100.57   
145000  105  290  289  109.77  414  107.62  2.15  0.00  102.96  403  100.65   
150000  104  300  299  109.75  414  107.59  2.16  0.00  102.96  403  100.66   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
210000  108  420  419  109.65  409  107.40  2.25  0.00  102.90  401  100.57   
215000  108  430  429  109.65  409  107.39  2.25  0.00  102.87  400  100.54   
220000  108  440  439  109.64  409  107.38  2.26  0.00  102.87  400  100.53   
225000  108  450  449  109.64  408  107.37  2.26  0.00  102.87  400  100.52   
230000  107  460  459  109.63  408  107.36  2.27  0.00  102.87  400  100.53   
235000  106  470  469  109.62  408  107.35  2.27  0.00  102.88  400  100.53   
240000  106  480  479  109.62  408  107.34  2.28  0.00  102.91  401  100.55   
245000  130  490  489  109.61  407  107.33  2.28  0.00  102.89  400  100.54   
250000  114  501    0  109.61  407  107.32  2.29  0.00  102.85  399  100.50   
255000  107  511   10  109.60  407  107.31  2.29  0.00  102.83  398  100.47   
260000  107  521   20  109.60  407  107.30  2.30  0.00  102.81  398  100.46   
265000  107  531   30  109.59  406  107.29  2.30  0.00  102.82  398  100.48   
270000   98  541   40  109.59  406  107.28  2.30  0.00  102.91  401  100.59   
275000  105  551   50  109.58  406  107.27  2.31  0.00  102.84  398  100.48   
280000  105  561   60  109.58  406  107.26  2.31  0.00  102.85  398  100.49   
285000   98  571   70  109.58  406  107.26  2.31  0.00  102.94  401  100.61   
290000  107  581   80  109.57  406  107.25  2.32  0.00  102.85  398  100.48   
295000   98  591   90  109.57  405  107.24  2.32  0.00  102.89  400  100.

0 R: 1.000 P: 0.323 carry pockets room back strap space shoulder work nice small
   1 R: 0.319 P: 0.110 ; & sleeve inch perfectly ! ipad inside pro perfect
     11 R: 0.066 P: 0.066 ! love bought ... recommend perfect buy price loves highly
     13 R: 0.056 P: 0.056 amazon item received shipping $ return ordered price arrived quality
     15 R: 0.037 P: 0.037 color picture pro blue pink ordered perfectly nice retina air
     14 R: 0.051 P: 0.051 cover keyboard color love mac perfectly pro hard apple easy
   3 R: 0.090 P: 0.035 - : zipper strap cards velcro flap pocket open main
     32 R: 0.055 P: 0.055 months zipper broke years quality bought year started handle 've
   5 R: 0.076 P: 0.047 power pocket mouse netbook cord drive small room usb charger
     51 R: 0.006 P: 0.006 $ ... ! leather cheap buy price quality thing material
     52 R: 0.023 P: 0.023 smell bad sleeve strong reviews days 'm smells fine air
   4 R: 0.192 P: 0.080 sleeve protection inside protect zipper pro padding ai

KeyboardInterrupt: 